In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from kernel_svm import *
from kernel_functions import *

In [2]:
train = pd.read_csv('../data/Xtr.csv', sep = ",", header = None)
trainOutput = pd.read_csv('../data/Ytr.csv', sep = ",")
test = pd.read_csv('../data/Xte.csv', sep = ",", header = None)

In [3]:
train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
test.drop(test.columns[len(test.columns)-1], axis=1, inplace=True)

In [4]:
class multiclassSVM:
    
    def __init__(self, K=None, kernel_fun=None, C=300, **kwargs):
        self.kernel_fun = kernel_fun
        self.parameters = kwargs
        self.C = C
        self.K = K
        self.models = {}
        self.classes = None

    def fit(self, X_train, y_train):
        
        classes = np.unique(y_train)
        self.classes = classes
        for l1 in range(len(classes)):
            for l2 in range(l1+1,len(classes)):
                self.models[classes[l1],classes[l2]] = KernelSVM(K=self.K, kernel_fun=self.kernel_fun, C=self.C)
                X_train_local = X_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = y_train[(y_train == classes[l1]) + (y_train == classes[l2])]
                y_train_local = 2*(y_train_local == classes[l1]) - 1
                self.models[classes[l1],classes[l2]].fit(X_train_local, y_train_local)
        
        
    def predict(self, X_test):
        
        ensemble_predictions = np.empty([len(X_test), 0])
        for k in self.models.keys():
            local_prediction = self.models[k].predict(X_test)
            ensemble_predictions = np.hstack((ensemble_predictions, np.array([k[0]*(local_prediction == 1) + k[1]*(local_prediction == -1)]).T))
            
        sum_array = np.array([np.sum(ensemble_predictions == c, axis = 1) for c in self.classes]).T
        final_predictions = self.classes[np.argmax(sum_array, axis=1)]
            
        return final_predictions, ensemble_predictions, sum_array

In [5]:
length = 100

ttrain = train[:length]
ttrainOutput = trainOutput[:length]['Prediction']
ttest = test[:length//2]

In [6]:
mSVM = multiclassSVM(kernel_fun = linear_kernel)

In [7]:
mSVM.fit(ttrain, ttrainOutput)

0
(14, 1)


/home/marc/anaconda3/lib/python3.5/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


ValueError: Rank(A) < p or Rank([P; A; G]) < n

In [ ]:
# for k, model in mSVM.models.items():
#     print(k, model.support_vectors_)

In [ ]:
final_predictions, ensemble_predictions, sum_array = mSVM.predict(ttest)

In [ ]:
ens_pred[0]

In [ ]:
sum_array[0]

In [ ]:
len(final_predictions)